In [5]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(r"C:\Users\super\DAG")
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
print(os.getcwd())
from SCM_data import generate_scm_from_BN 
from numpy.linalg import inv
from scipy.linalg import sqrtm

c:\Users\super\DAG\golem-main\src


In [1]:
import os
import numpy as np

from models import GolemModel
from trainers import GolemTrainer
from data_loader import SyntheticDataset
from data_loader import SCM_data

from utils import MEC


# For logging of tensorflow messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


def golem(X, lambda_1, lambda_2, equal_variances=True,
          num_iter=1e+5, learning_rate=1e-3, seed=1,
          checkpoint_iter=None, output_dir=None, B_init=None):
    """Solve the unconstrained optimization problem of GOLEM, which involves
        GolemModel and GolemTrainer.

    Args:
        X (numpy.ndarray): [n, d] data matrix.
        lambda_1 (float): Coefficient of L1 penalty.
        lambda_2 (float): Coefficient of DAG penalty.
        equal_variances (bool): Whether to assume equal noise variances
            for likelibood objective. Default: True.
        num_iter (int): Number of iterations for training.
        learning_rate (float): Learning rate of Adam optimizer. Default: 1e-3.
        seed (int): Random seed. Default: 1.
        checkpoint_iter (int): Number of iterations between each checkpoint.
            Set to None to disable. Default: None.
        output_dir (str): Output directory to save training outputs.
        B_init (numpy.ndarray or None): [d, d] weighted matrix for initialization.
            Set to None to disable. Default: None.

    Returns:
        numpy.ndarray: [d, d] estimated weighted matrix.

    Hyperparameters:
        (1) GOLEM-NV: equal_variances=False, lambda_1=2e-3, lambda_2=5.0.
        (2) GOLEM-EV: equal_variances=True, lambda_1=2e-2, lambda_2=5.0.
    """
    # Center the data
    X = X - X.mean(axis=0, keepdims=True)

    # Set up model
    n, d = X.shape
    model = GolemModel(n, d, lambda_1, lambda_2, equal_variances, seed, B_init)

    # Training
    trainer = GolemTrainer(learning_rate)
    B_est = trainer.train(model, X, num_iter, checkpoint_iter, output_dir)

    return B_est    # Not thresholded yet

def weight_to_adjacency(W, threshold=0.05):
    """
    Convert a weight matrix to an adjacency matrix.
    
    Parameters:
        W (np.ndarray): Weight matrix (square matrix).
        threshold (float): Values with absolute weight <= threshold are treated as 0.
    
    Returns:
        np.ndarray: Binary adjacency matrix of the same shape.
    """
    if not isinstance(W, np.ndarray):
        raise TypeError("Input W must be a numpy array.")
    if W.shape[0] != W.shape[1]:
        raise ValueError("Input W must be a square matrix.")
    
    G = (np.abs(W) > threshold).astype(int)
    return G


In [3]:
import logging

from data_loader import SyntheticDataset
from data_loader.synthetic_dataset import dataset_based_on_B
from utils.train import postprocess
from utils.utils import count_accuracy, set_seed

# Setup for logging
# Required for printing histories if checkpointing is activated
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s - %(name)s - %(message)s'
)

# Reproducibility
set_seed(1)

# Load dataset
n, d = 1000, 4
graph_type, degree = 'ER', 0.5    # ER2 graph
B_scale = 1.0
noise_type = 'gaussian_ev'
#dataset = SyntheticDataset(n, d, graph_type, degree,
#                           noise_type, B_scale, seed=1)

'''
times = 20
for i in range(1, 6):
    true_count = [0] * 6
    for seed in range(times):
        X, Y, Z, G_true, CPDAG = SCM_data.generate_scm_data(i,10000, seed = seed)
        data = np.array([X, Y, Z]).T
        #print(data.T@ data / 10000)
        W_est = golem(data, lambda_1=2e-2, lambda_2=5.0,
                equal_variances=True, num_iter=1e+4)
        G_est = weight_to_adjacency(W_est, 0.05)
        if MEC.is_in_markov_equiv_class(G_true, G_est): true_count[i-1] += 1
    print(f"SCM {i} : {true_count[i-1]/times}")
'''

for i in range(1, 6):
    X, Y, Z, G_true, CPDAG = SCM_data.generate_scm_data(i,10000, seed = 1)
    data = np.array([X, Y, Z]).T
    W_est = golem(data, lambda_1=2e-2, lambda_2=5.0,
                equal_variances=False, num_iter=1e+4)
    G_est = weight_to_adjacency(W_est, 0.05)
    print("pattern",i)
    print("G_true = \n",G_true)
    print("G_est = \n",G_est)

2025-11-25 23:09:53,420 INFO - trainers.golem_trainer - Started training for 10000.0 iterations.


pattern 1
G_true = 
 [[0 0 0]
 [0 0 0]
 [0 0 0]]
G_est = 
 [[0 0 0]
 [0 0 0]
 [0 0 0]]


2025-11-25 23:10:04,576 INFO - trainers.golem_trainer - Started training for 10000.0 iterations.


pattern 2
G_true = 
 [[0 1 0]
 [0 0 0]
 [0 0 0]]
G_est = 
 [[0 0 0]
 [1 0 0]
 [0 0 0]]


2025-11-25 23:10:16,495 INFO - trainers.golem_trainer - Started training for 10000.0 iterations.


pattern 3
G_true = 
 [[0 1 0]
 [0 0 0]
 [0 1 0]]
G_est = 
 [[0 0 0]
 [1 0 1]
 [1 0 0]]


2025-11-25 23:10:27,384 INFO - trainers.golem_trainer - Started training for 10000.0 iterations.


pattern 4
G_true = 
 [[0 1 0]
 [0 0 1]
 [0 0 0]]
G_est = 
 [[0 1 0]
 [0 0 0]
 [1 1 0]]


2025-11-25 23:10:39,071 INFO - trainers.golem_trainer - Started training for 10000.0 iterations.


pattern 5
G_true = 
 [[0 1 1]
 [0 0 1]
 [0 0 0]]
G_est = 
 [[0 1 0]
 [0 0 0]
 [1 1 0]]


In [ ]:
# A->B, B->C, B->D, |v| = 0
print("d=4, A->B, B->C, B->D, |v| = 0")
B_true = np.array([
    [0, 2, 0, 0],
    [0, 0, 3, 4],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
])
N = [1, 4, 3, 2]


data, G, B, Sigma = generate_scm_from_BN(B_true.T, n_samples=10000, N=N, seed=42)
W_est = golem(data, lambda_1=2e-2, lambda_2=5.0,
            equal_variances=False, num_iter=1e+4)
G_est = weight_to_adjacency(W_est, 0.05)
print("pattern",i)
print("G_true = \n",B_true)
print("G_est = \n",G_est)


# A->C, A->D, B->C, B->D, |v| = 2
print("d=4, A->C, A->D, B->C, B->D, |v| = 2")
B_true = np.array([
    [0, 0, 2, 3],
    [0, 0, 3, 4],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
])
N = [2, 4, 3, 5]

data, G, B, Sigma = generate_scm_from_BN(B_true.T, n_samples=10000, N=N, seed=42)
W_est = golem(data, lambda_1=2e-2, lambda_2=5.0,
            equal_variances=False, num_iter=1e+4)
G_est = weight_to_adjacency(W_est, 0.05)
print("pattern",i)
print("G_true = \n",B_true)
print("G_est = \n",G_est)

# A->D, B->D, C->D, |v| = 3
print("d=4, A->D, B->D, C->D, |v| = 3")

B_true = np.array([
    [0, 0, 0, 1],
    [0, 0, 0, 3],
    [0, 0, 0, 5],
    [0, 0, 0, 0]
])
N = [5, 4, 3, 2]

data, G, B, Sigma = generate_scm_from_BN(B_true.T, n_samples=10000, N=N, seed=42)
W_est = golem(data, lambda_1=0.1, lambda_2=5.0,
            equal_variances=False, num_iter=1e+4)
G_est = weight_to_adjacency(W_est, 0.05)
print("pattern",i)
print("G_true = \n",B_true)
print("G_est = \n",G_est)

d=4, A->B, B->C, B->D, |v| = 0


2025-11-25 23:24:02,258 INFO - trainers.golem_trainer - Started training for 10000.0 iterations.


pattern 1
G_true = 
 [[0 2 0 0]
 [0 0 3 4]
 [0 0 0 0]
 [0 0 0 0]]
G_est = 
 [[0 0 0 0]
 [0 0 0 0]
 [0 1 0 0]
 [0 1 1 0]]
d=4, A->C, A->D, B->C, B->D, |v| = 2


2025-11-25 23:24:13,946 INFO - trainers.golem_trainer - Started training for 10000.0 iterations.


pattern 1
G_true = 
 [[0 0 2 3]
 [0 0 3 4]
 [0 0 0 0]
 [0 0 0 0]]
G_est = 
 [[0 1 0 0]
 [0 0 0 0]
 [0 1 0 0]
 [1 1 1 0]]
d=4, A->D, B->D, C->D, |v| = 3


2025-11-25 23:24:25,976 INFO - trainers.golem_trainer - Started training for 100000.0 iterations.


pattern 1
G_true = 
 [[0 0 0 1]
 [0 0 0 3]
 [0 0 0 5]
 [0 0 0 0]]
G_est = 
 [[0 1 1 0]
 [0 0 1 0]
 [0 0 0 0]
 [0 1 1 0]]


In [17]:
# A->B, B->C, B->D, |v| = 0
print("d=4, A->B, B->C, B->D, |v| = 0")
B_true = np.array([
    [0, 2, 0, 0],
    [0, 0, 3, 4],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
])
N = [1, 4, 3, 2]


data, G, B, Sigma = generate_scm_from_BN(B_true.T, n_samples=10000, N=N, seed=42)
W_est = golem(data, lambda_1=0.2, lambda_2=5.0,
            equal_variances=True, num_iter=1e+4)
G_est = weight_to_adjacency(W_est, 0.05)
print("pattern",i)
print("G_true = \n",B_true)
print("G_est = \n",G_est)


# A->C, A->D, B->C, B->D, |v| = 2
print("d=4, A->C, A->D, B->C, B->D, |v| = 2")
B_true = np.array([
    [0, 0, 2, 3],
    [0, 0, 3, 4],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
])
N = [2, 4, 3, 5]

data, G, B, Sigma = generate_scm_from_BN(B_true.T, n_samples=10000, N=N, seed=42)
W_est = golem(data, lambda_1=0.2, lambda_2=5.0,
            equal_variances=True, num_iter=1e+4)
G_est = weight_to_adjacency(W_est, 0.05)
print("pattern",i)
print("G_true = \n",B_true)
print("G_est = \n",G_est)

# A->D, B->D, C->D, |v| = 3
print("d=4, A->D, B->D, C->D, |v| = 3")

B_true = np.array([
    [0, 0, 0, 1],
    [0, 0, 0, 3],
    [0, 0, 0, 5],
    [0, 0, 0, 0]
])
N = [5, 4, 3, 2]

data, G, B, Sigma = generate_scm_from_BN(B_true.T, n_samples=10000, N=N, seed=42)
W_est = golem(data, lambda_1=0.2, lambda_2=5.0,
            equal_variances=True, num_iter=1e+4)
G_est = weight_to_adjacency(W_est, 0.05)
print("pattern",i)
print("G_true = \n",B_true)
print("G_est = \n",G_est)

d=4, A->B, B->C, B->D, |v| = 0


2025-11-25 23:29:01,268 INFO - trainers.golem_trainer - Started training for 10000.0 iterations.


pattern 1
G_true = 
 [[0 2 0 0]
 [0 0 3 4]
 [0 0 0 0]
 [0 0 0 0]]
G_est = 
 [[0 1 0 0]
 [0 0 1 1]
 [0 0 0 0]
 [0 0 1 0]]
d=4, A->C, A->D, B->C, B->D, |v| = 2


2025-11-25 23:29:14,020 INFO - trainers.golem_trainer - Started training for 10000.0 iterations.


pattern 1
G_true = 
 [[0 0 2 3]
 [0 0 3 4]
 [0 0 0 0]
 [0 0 0 0]]
G_est = 
 [[0 0 1 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 0]]
d=4, A->D, B->D, C->D, |v| = 3


2025-11-25 23:29:26,517 INFO - trainers.golem_trainer - Started training for 10000.0 iterations.


pattern 1
G_true = 
 [[0 0 0 1]
 [0 0 0 3]
 [0 0 0 5]
 [0 0 0 0]]
G_est = 
 [[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 0]]


In [10]:
times = 20
for i in range(1, 6):
    true_count = [0] * 6
    for seed in range(times):
        X, Y, Z, G_true, CPDAG = SCM_data.generate_scm_data(i,10000, seed = seed)
        data = np.array([X, Y, Z]).T
        #print(data.T@ data / 10000)
        W_est = golem(data, lambda_1=2e-2, lambda_2=5.0,
                equal_variances=False, num_iter=1e+4)
        G_est = weight_to_adjacency(W_est, 0.05)
        if MEC.is_in_markov_equiv_class(G_true, G_est): true_count[i-1] += 1
    print(f"SCM {i} : {true_count[i-1]/times}")

2025-11-25 23:19:16,280 INFO - trainers.golem_trainer - Started training for 10000.0 iterations.
2025-11-25 23:19:28,261 INFO - trainers.golem_trainer - Started training for 10000.0 iterations.
2025-11-25 23:19:40,138 INFO - trainers.golem_trainer - Started training for 10000.0 iterations.


KeyboardInterrupt: 